# Анализ активности пользователей облачного гейминг-сервиса

## Описание проекта
Анализ данных по игровым сессиям пользователей облачного гейминг-сервиса за несколько дней

**Цель:** рассчитать ключевые метрики активности для понимания поведения пользователей

### Рассчитываемые метрики:
1. DAU (число уникальных пользователей в день)
2. Количество сессий для каждой платформы (PC/Android/iOS)
3. Средняя длительность сессии по платформам
4. Доля сессий новых пользователей
5. Медиана длительности сессий для новых и старых пользователей
6. Суммарное время в игре для пользователей из России

## 1. Загрузка и подготовка данных

In [20]:
# Импортируем необходимые библиотеки
import pandas as pd
import sqlite3

# Загружаем данные из CSV файла
test_df = pd.read_csv('data/test_dataset.csv')

# Выводим первые 5 строк для ознакомления со структурой
test_df.head()

,session_id,user_id,is_new_user,country,session_start,session_end,platform,game_id,session_status
0,1,328,True,RU,2025-01-01 1:06:17,2025-01-01 1:27:17,PC,102,completed
1,2,17,False,RU,2025-01-01 0:21:14,2025-01-01 2:36:14,PC,202,completed
2,3,302,True,US,2025-01-01 1:28:51,2025-01-01 2:27:51,Android,103,completed
3,4,391,True,NaN,2025-01-01 0:53:24,2025-01-01 1:34:24,IOS,105,crashed
4,5,374,True,US,2025-01-01 2:02:24,2025-01-01 2:09:24,IOS,201,completed


In [21]:
# Проверяем информацию о датасете: типы данных, пропуски
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   session_id      2000 non-null   int64 
 1   user_id         2000 non-null   int64 
 2   is_new_user     1703 non-null   object
 3   country         1591 non-null   object
 4   session_start   2000 non-null   object
 5   session_end     2000 non-null   object
 6   platform        2000 non-null   object
 7   game_id         2000 non-null   int64 
 8   session_status  2000 non-null   object
dtypes: int64(3), object(6)
memory usage: 140.8+ KB


### 1.1 Обработка дат
Приводим столбцы `session_start` и `session_end` к единому формату datetime

**Важно:**
- Исходные данные могут содержать разные форматы дат
- Параметр `errors='coerce'` преобразует некорректные значения в NaT (Not a Time)
- `utc=True` приводит все даты к единому часовому поясу для корректных расчётов

In [22]:
# Конвертируем строки в datetime с учётом часового пояса
test_df["session_start"] = pd.to_datetime(
    test_df["session_start"],
    errors="coerce",  # некорректные значения станут NaT
    utc=True
)

test_df["session_end"] = pd.to_datetime(
    test_df["session_end"],
    errors="coerce",
    utc=True
)

In [23]:
# Приводим к строковому формату для корректной работы в SQLite
# SQLite не имеет встроенного типа datetime, поэтому храним как строку
test_df["session_start"] = test_df["session_start"].dt.strftime("%Y-%m-%d %H:%M:%S")
test_df["session_end"]   = test_df["session_end"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [24]:
# Создаём подключение к SQLite
db_con = sqlite3.connect('test.db')

# Загружаем датафрейм в таблицу 'test'
# if_exists='replace' - перезаписываем таблицу, если она существует
test_df.to_sql('test', db_con, if_exists='replace', index=False)

2000

## 2. Расчёт метрик
### 2.1 Число уникальных пользователей в день (DAU)

**Логика расчёта:**
- Группируем данные по дате начала сессии
- Считаем количество уникальных `user_id` за каждый день
- Сортируем по убыванию для наглядности

In [25]:
dau = """
    SELECT
        strftime('%Y-%m-%d', session_start) AS day,  -- извлекаем дату из timestamp
        COUNT(DISTINCT user_id) AS count_users       -- считаем уникальных пользователей
    FROM test
    GROUP BY day
    ORDER BY count_users DESC
"""

pd.read_sql(dau, db_con)

,day,count_users
0,2025-01-01,417
1,2025-01-02,90
2,2025-01-03,1


### 2.2 Количество игровых сессий на каждой платформе (PC/Android/iOS)

**Логика расчёта:**
- Группируем по полю `platform`
- Считаем количество сессий для каждой группы

In [26]:
count_session = """
    SELECT
        platform,                        -- платформа
        COUNT(session_id) AS session        -- количество сессий
    FROM test
    GROUP BY platform
    ORDER BY session DESC
"""

pd.read_sql(count_session, db_con)

,platform,session
0,IOS,688
1,PC,657
2,Android,655


### 2.3 Средняя продолжительность игровой сессии для каждой платформы

**Логика расчёта:**
- Вычисляем длительность сессии: `session_end - session_start`
- `strftime('%s', ...)` конвертирует datetime в секунды
- Делим на 60 для перевода в минуты
- Применяем функцию AVG для каждой платформы

In [27]:
avg_session = """
    SELECT
        platform,
        AVG(
            (strftime('%s', session_end) - strftime('%s', session_start)) / 60.0
        ) AS avg_session  -- средняя длительность в минутах
    FROM test
    GROUP BY platform
    ORDER BY avg_session DESC
"""

pd.read_sql(avg_session, db_con)

,platform,avg_session
0,IOS,47.344477
1,PC,43.773212
2,Android,42.867176


### 2.4 Доля сессий, которая приходится на новых пользователей

**Логика расчёта:**
- Числитель: количество сессий, где `is_new_user = True`
- Знаменатель: общее количество сессий, где `is_new_user IS NOT NULL`
- Значения NULL исключаются из расчёта согласно заданию
- Результат умножаем на 100 для получения процента

In [28]:
percent = """
SELECT
    ROUND(
        100.0 *
        (SELECT COUNT(session_id)
         FROM test
         WHERE is_new_user IS True AND is_new_user IS NOT NULL)  -- сессии новых пользователей
        /
        (SELECT COUNT(session_id)
         FROM test
         WHERE is_new_user IS NOT NULL),  -- все сессии с известным статусом
    2) AS dolya_percent
"""

pd.read_sql(percent, db_con)

,dolya_percent
0,29.71


### 2.5 Общее время, проведённое в играх пользователями из России

In [29]:
total_time_ru = """
SELECT
    ROUND(
        SUM(
            (strftime('%s', session_end) - strftime('%s', session_start)) / 3600.0
        ),
    2) AS sum_in_hours  -- суммарное время в часах
FROM test
WHERE country = 'RU'
"""

pd.read_sql(total_time_ru, db_con)

,sum_in_hours
0,359.82


### 2.6 Медиана длительности сессий отдельно для новых и старых пользователей

**Логика расчёта:**
- SQLite не имеет встроенной функции MEDIAN
- Поэтому выгружаем данные в pandas и используем `.median()`

In [30]:
# Выгружаем данные по новым пользователям
new_users_df = pd.read_sql("""
    SELECT
        session_id,
        (strftime('%s', session_end) - strftime('%s', session_start)) / 60.0 AS time_new
    FROM test
    WHERE is_new_user IS True AND is_new_user IS NOT NULL
""", db_con)

# Отображаем данные
new_users_df

,session_id,time_new
0,1,21.0
1,3,59.0
2,4,41.0
3,5,7.0
4,9,41.0
...,...,...
501,1981,9.0
502,1990,58.0
503,1997,141.0
504,1999,2.0


In [31]:
# Медиана длительности сессий у НОВЫХ пользователей
median_new = new_users_df['time_new'].median()
print(f"Медиана длительности сессий (новые пользователи): {median_new:.2f} минут")

Медиана длительности сессий (новые пользователи): 31.50 минут


подсчитана медиана длительности сессиий у новых пользователей

In [32]:
new_users_df['time_new'].median()

31.5

In [33]:
# Выгружаем данные по старым пользователям
old_users_df = pd.read_sql("""
    SELECT
        session_id,
        (strftime('%s', session_end) - strftime('%s', session_start)) / 60.0 AS time_old
    FROM test
    WHERE is_new_user IS False AND is_new_user IS NOT NULL
""", db_con)

# Отображаем данные
old_users_df

,session_id,time_old
0,2,135.0
1,6,7.0
2,7,2.0
3,8,90.0
4,10,55.0
...,...,...
1192,1993,42.0
1193,1994,6.0
1194,1995,62.0
1195,1996,48.0


In [34]:
# Медиана длительности сессий у СТАРЫХ пользователей
median_old = old_users_df['time_old'].median()
print(f"Медиана длительности сессий (старые пользователи): {median_old:.2f} минут")

Медиана длительности сессий (старые пользователи): 32.00 минут


## 3. Вывод
Все метрики рассчитывались с использованием библиотеки **Pandas** и **SQLite**

### Обработка данных:
- Даты приведены к единому формату времени, некорректные значения преобразованы в NaT
- NULL исключены из расчёта доли по заданию

### Возможные дополнительные метрики:
- Retention Rate (возвращаемость пользователей)
- Конверсия из новых в активных пользователей

### Интерпретация результатов
* Пользователи **iOS** проводят в игре больше времени (в среднем 47 мин), чем пользователи Android и PC, несмотря на схожее количество сессий. Это может указывать на лучшую оптимизацию игры под iOS или особенности аудитории.
* Доля сессий от **новых пользователей** составляет около 30%, что говорит о значительном притоке аудитории.
* Медианная длительность сессии у "старичков" и "новичков" практически не отличается (~31-32 мин), что говорит о том, что игра затягивает с первых сессий